In [1]:
import shutil

# importing the sys module
import sys        

# appending the directory of mod.py
# in the sys.path list
sys.path.append('../')   

import json
import os
import random
from collections import defaultdict

import torch
from PIL import Image
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import models
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torch.optim import lr_scheduler
from torch.optim import Adam
import numpy as np
from sklearn.model_selection import train_test_split

from datetime import datetime


import diff_pipeline4__config_diff_infer as cfg

from diff_pipeline4__dataset_rich import DiffPipeline4SameUniformDiffDataset, DiffPipeline4MixedUniformDiffDataset, DiffPipeline4MimicDiffDataset, get_error_constrained_dataset


from Common.Datasets.Morph2.data_parser import DataParser
from diff_pipeline4__train import train_diff_cls_model_iter


from Common.Optimizers.RangerLars import RangerLars
from Common.Schedulers.GradualWarmupScheduler import GradualWarmupScheduler


from diff_pipeline4__model import DiffPipeline4Model, DiffPipeline4DeepModel, AgeDiffModel, DiffModelConfigType

from Common.Pipelines.AgePredictBasicPipeline import AgePredictBasicPipeline 
from Common.Pipelines.AgePredictSingleRefPipeline import AgePredictSingleRefEnhancedPipeline
from Common.Analysis.DiffAnalysisMethods import diff_pipeline_confusion_matrix_analysis


from tqdm import tqdm


from Common.Models.UnifiedClassificaionAndRegressionAgeModel import UnifiedClassificaionAndRegressionAgeModel
from Common.Models.transformer import *
from Common.Models.unified_transformer_model import AgeTransformer

from sklearn import utils

if __name__ == "__main__":

    torch.manual_seed(cfg.RANDOM_SEED)
    np.random.seed(cfg.RANDOM_SEED)
    random.seed(cfg.RANDOM_SEED)

    if cfg.USE_GPU:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    else:
        device = torch.device("cpu")

    print(device)

    torch.cuda.empty_cache()





    # Load data
    data_parser = DataParser('../Common/Datasets/Morph2/aligned_data/aligned_dataset_with_metadata_uint8.hdf5', small_data=cfg.SMALL_DATA)
    data_parser.initialize_data()

    x_train, y_train, x_test, y_test = data_parser.x_train,	data_parser.y_train, data_parser.x_test, data_parser.y_test,
    if cfg.RANDOM_SPLIT:
        all_images = np.concatenate((x_train, x_test), axis=0)
        all_labels = np.concatenate((y_train, y_test), axis=0)

        x_train, x_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=cfg.TEST_SIZE_FOR_RS, random_state=cfg.RANDOM_SEED)

    
    face2emb_arr_trn_r = np.load('face2emb_arr_trn.npy', allow_pickle=True)
#     import pdb
#     pdb.set_trace()
    #face2emb_arr_trn_r = face2emb_arr_trn_r.item()
    
    face2emb_arr_vld_r = np.load('face2emb_arr_vld.npy', allow_pickle=True)
    #face2emb_arr_vld_r = face2emb_arr_vld_r.item()
    
    # shuffle basic aligned test
    # x_test_shuffled
    # y_test_shuffled


    with open("im2age_map_test.json", 'r') as im2age_map_test_f:
        im2age_map_test = json.load(im2age_map_test_f)

    # with open("im2age_map_test_from_actual_pipe.json", 'r') as im2age_map_test_f:
    # 	im2age_map_test = json.load(im2age_map_test_f)



    # 	transf = transforms.Compose([
    #             transforms.RandomResizedCrop(224, (0.9, 1.0)),
    #             transforms.RandomHorizontalFlip(),
    #             transforms.RandomApply([transforms.ColorJitter(
    #                 brightness=0.1,
    #                 contrast=0.1,
    #                 saturation=0.1,
    #                 hue=0.1
    #             )], p=0.5),
    #             transforms.RandomApply([transforms.RandomAffine(
    #                 degrees=10,
    #                 translate=(0.1, 0.1),
    #                 scale=(0.9, 1.1),
    #                 shear=5,
    #                 resample=Image.BICUBIC
    #             )], p=0.5),
    #             transforms.ToTensor(),
    #             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #             transforms.RandomErasing(p=0.5)
    #         ])

    face2emb_arr_trn_r = np.load('face2emb_arr_trn.npy', allow_pickle=True)
#     import pdb
#     pdb.set_trace()
    #face2emb_arr_trn_r = face2emb_arr_trn_r.item()
    
    face2emb_arr_vld_r = np.load('face2emb_arr_vld.npy', allow_pickle=True)
    
#     transf = transforms.Compose([
#             transforms.Resize(224), # just for testing
#     #          transforms.RandomResizedCrop(224, (0.9, 1.0)),
#     #          transforms.RandomHorizontalFlip(),
#             transforms.RandomApply([transforms.ColorJitter(
#                 brightness=0.1,
#                 contrast=0.1,
#                 saturation=0.1,
#                 hue=0.1
#             )], p=0.5),
#     #         transforms.RandomApply([transforms.RandomAffine(
#     #             degrees=10,
#     #             translate=(0.1, 0.1),
#     #             scale=(0.9, 1.1),
#     #             shear=5,
#     #             resample=Image.BICUBIC
#     #         )], p=0.5),
#             transforms.ToTensor(),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     #        transforms.RandomErasing(p=0.5)
#         ])

    transf = transforms.Compose([
        transforms.RandomResizedCrop(224, (0.9, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomApply([transforms.ColorJitter(
            brightness=0.1,
            contrast=0.1,
            saturation=0.1,
            hue=0.1
        )], p=0.5),
        transforms.RandomApply([transforms.RandomAffine(
            degrees=10,
            translate=(0.1, 0.1),
            scale=(0.9, 1.1),
            shear=5,
            resample=Image.BICUBIC
        )], p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.5)
    ])


    train_ds = DiffPipeline4SameUniformDiffDataset(
        data_set_images=x_train,
        data_set_metadata=y_train,
        min_age=cfg.MIN_AGE,
        age_interval=cfg.AGE_INTERVAL,
        max_age=cfg.MAX_AGE,
        transform=transf,
        copies=1, 
        age_radius=cfg.AGE_RADIUS,
        age_diff_learn_radius_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO,
        age_diff_learn_radius_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI,
        embs=face2emb_arr_trn_r#,
        #num_references=cfg.NUM_OF_REFS
    )
    
    
    x_test_filtered, y_test_filtered, batst_set_filtered_indexes = get_error_constrained_dataset(orig_dataset_images=x_test, 
                                                                                                    orig_dataset_metadata=y_test,
                                                                                                    age_diff_learn_radius_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO,
                                                                                                    age_diff_learn_radius_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI,
                                                                                                    im2age_map_batst=im2age_map_test)

    # apref = Age Predict Reference (dataset is based on (q,r) pairs where r's age is what AgePredict model returns on q)
    test_ds = DiffPipeline4MimicDiffDataset(
        batrn_set_images=x_train,
        batrn_set_metadata=y_train,
        batst_set_images=x_test_filtered,
        batst_set_metadata=y_test_filtered,
        batst_set_indexes=batst_set_filtered_indexes,
        im2age_map_batst=im2age_map_test,
        min_age=cfg.MIN_AGE,
        age_interval=cfg.AGE_INTERVAL,
        max_age=cfg.MAX_AGE,
        transform=transf,
        copies=1, 
        age_radius=cfg.AGE_RADIUS,
        embs_trn=face2emb_arr_trn_r,
        embs_vld=face2emb_arr_vld_r
    )
    
    

    print("Testing (q tst where AgePredict(q)  {age_diff_learn_radius_lo} <= error <= {age_diff_learn_radius_hi}, r trn where age(r)=AgePredict(q)) set size: ".format(age_diff_learn_radius_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO, age_diff_learn_radius_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI) + str(len(test_ds)))

    
    test_qtst_rtst_ds = DiffPipeline4SameUniformDiffDataset(
        data_set_images=x_test,
        data_set_metadata=y_test,
        min_age=cfg.MIN_AGE,
        age_interval=cfg.AGE_INTERVAL,
        max_age=cfg.MAX_AGE,
        transform=transf,
        copies=1, 
        age_radius=cfg.AGE_RADIUS,
        age_diff_learn_radius_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO,
        age_diff_learn_radius_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI,
        embs=face2emb_arr_vld_r#,
        #num_references=cfg.NUM_OF_REFS
    )
    
    test_qtst_rtrn_ds = DiffPipeline4MixedUniformDiffDataset(
        batrn_set_images=x_train,
        batrn_set_metadata=y_train,
        batst_set_images=x_test,
        batst_set_metadata=y_test,
        min_age=cfg.MIN_AGE,
        age_interval=cfg.AGE_INTERVAL,
        max_age=cfg.MAX_AGE,
        transform=transf,
        copies=1, 
        age_radius=cfg.AGE_RADIUS,
        age_diff_learn_radius_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO,
        age_diff_learn_radius_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI,
        embs_trn=face2emb_arr_trn_r,
        embs_vld=face2emb_arr_vld_r#,
        #num_references=cfg.NUM_OF_REFS
    )
    

    image_datasets = {
        'train': train_ds,
        'val': test_ds
        #'val': test_qtst_rtrn_ds
        #'val': test_qtst_rtst_ds
    }

    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

    data_loaders = {
        #'train': DataLoader(train_ds, batch_size=cfg.BATCH_SIZE, num_workers=cfg.NUM_OF_WORKERS_DATALOADER, pin_memory=True, shuffle=True, drop_last=True),
        'val': DataLoader(test_ds, batch_size=cfg.BATCH_SIZE, num_workers=cfg.NUM_OF_WORKERS_DATALOADER, pin_memory=True, shuffle=False, drop_last=True)
        #'val': DataLoader(test_qtst_rtrn_ds, batch_size=cfg.BATCH_SIZE, num_workers=cfg.NUM_OF_WORKERS_DATALOADER, pin_memory=True, shuffle=False, drop_last=True)
        #'val': DataLoader(test_qtst_rtst_ds, batch_size=cfg.BATCH_SIZE, num_workers=cfg.NUM_OF_WORKERS_DATALOADER, pin_memory=True, shuffle=False, drop_last=True)
    }


    num_classes = int((cfg.MAX_AGE - cfg.MIN_AGE) / cfg.AGE_INTERVAL + 1)

    if cfg.DEEP_MODEL:
        #model = DiffPipeline4DeepModel
        model = AgeDiffModel(age_interval=cfg.AGE_INTERVAL, 
                                min_age=cfg.MIN_AGE, 
                                max_age=cfg.MAX_AGE, 
                                age_radius=cfg.AGE_RADIUS, 
                                device=device, 
                                deep=True, 
                                num_references=1,#cfg.NUM_OF_REFERENCES, 
                                config_type=DiffModelConfigType.AddedEmbeddingAndMlpHeadWithDiffHead, 
                                added_embed_layer_size=512, 
                                diff_embed_layer_size=512,
                                is_ordinal_reg=True)
    else:
        model = DiffPipeline4Model


    print("testing set size: " + str(len(test_ds)))

    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_23_09_2022_20_10_15'
    #model_filename = 'weights.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_30_10_2022_17_23_12'
    #model_filename = 'weights_11064_1.3333.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_22_11_2022_01_18_41'
    #model_filename = 'weights_6915_1.9282.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_31_10_2022_20_48_14'
    #model_filename = 'weights_12447_1.4136.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_24_11_2022_00_06_22'
    #model_filename = 'weights_544_1.3960.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_24_11_2022_02_08_36'
    #model_filename = 'weights_12600_1.6146.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_24_11_2022_10_39_35'
    #model_filename = 'weights_34200_1.5485.pt' #'weights_15600_1.6087.pt' #'weights_16596_1.5844.pt' #'weights_34200_1.5485.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_26_11_2022_19_26_33'
    #model_filename = 'weights_19362_1.6191.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_30_10_2022_17_23_12' # trained in range 0-3 with close embs (from 25 percentile), unfreeze backbone in epoch 5
    #model_filename = 'weights_11064_1.3333.pt'
    
    model_dir = 'F:/DiffPipeline4/time_04_12_2022_09_07_53'
    model_filename = 'weights_80214_2.8404.pt'
    
    pipeline = AgePredictSingleRefEnhancedPipeline(
                        device,
                        num_classes, 
                        cfg.AGE_INTERVAL, 
                        cfg.MIN_AGE, 
                        cfg.MAX_AGE, 
                        cfg.AGE_RADIUS,
                        0, #cfg.MID_FEATURE_SIZE_AGE_PREDICT, 
                        x_train,
                        y_train,
                        0, #cfg.DEEP, 
                        0, #cfg.NUM_OF_REFERENCES, 
                        0, #cfg.DIFF_MODEL_CONFIG_TYPE, 
                        0, #cfg.ADDED_EMBED_LAYER_SIZE,
                        0, #cfg.DIFF_EMBED_LAYER_SIZE,
                        transf,
                        0, #cfg.NUM_OF_COPIES_AGE_PREDICT,
                        UnifiedClassificaionAndRegressionAgeModel,
                        TransformerModel,
                        AgeTransformer,
                        model, #AgeDiffModelNonOpt, #AgeDiffModel,
                        True, #cfg.IS_ORDINAL_REG,
                        no_age_transformer_init=True,
                        cfg=cfg,
                        age_diff_model_path_arg=model_dir,
                        age_diff_model_file_name_arg=model_filename,
                        embs_trn=face2emb_arr_trn_r,
                        embs_vld=face2emb_arr_vld_r)


    diff_pipeline_confusion_matrix_analysis(pipeline, data_loaders['val'], device, dataset_sizes['val'], is_ordinal_reg=True)
    #diff_pipeline_confusion_matrix_analysis(pipeline, data_loaders['train'], device, dataset_sizes['train'], is_ordinal_reg=True)



cuda:0


C:\Users\proj\.conda\envs\my_thesis_py38\lib\site-packages\torchvision\transforms\transforms.py:1361: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  warnings.warn(


Testing (q tst where AgePredict(q)  0 <= error <= 35, r trn where age(r)=AgePredict(q)) set size: 10606
testing set size: 10606
Using multiple GPUs (2)


  0%|                                                                                          | 0/331 [00:00<?, ?it/s]

running on validation set...


 90%|███████████████████████████████████████████████████████████████████████▊        | 297/331 [02:38<00:18,  1.87it/s]


KeyboardInterrupt: 

In [ ]:
pipeline